In [1]:
# -*-  coding: utf-8 -*-
import os, sys
import jieba, codecs, math
import jieba.posseg as pseg

names = {}            # 字典
relationships = {}    # 关系字典
lineNames = []        # 关系

In [5]:
jieba.load_userdict("ehikoo-dict.txt")        # 加载字典
with codecs.open("ehikoo201707.txt", "r", "GB2312") as f:
    for line in f.readlines():
        poss = pseg.cut(line)        # 分词并返回该词词性
        lineNames.append([])        # 为新读入的一段添加关键词名称列表
        for w in poss:
            if w.flag != "nr" or len(w.word) < 2:
                continue            # 当分词长度小于2或该词词性不为nr时认为该词不为关键词
            lineNames[-1].append(w.word)        # 为当前段的环境增加一个关键词
            if names.get(w.word) is None:
                names[w.word] = 0
                relationships[w.word] = {}
            names[w.word] += 1                    # 该关键词出现次数加 1

In [6]:
for name, times in names.items():
    print(name, times) #打印关键词出现的次数

行业 16
企业 38
航运 42
建造 40
船舶 94
风险 48
市场 14
发展 22
公司 26
李克强 6
集装箱 20
马士基 6
达飞 2
长荣 2
规范 68
数据 34
全球 24
生效 28
技术 40
胡问鸣 2
孙波 2
邵开文 2
杜刚 2
凯旋 2
罗罗 2
罗尔斯 2
罗伊斯 2
公约 48
内容简介 2
检验 64
要求 42
罗毕 2
指南 100
奥氏体 4
于长江 2


In [7]:
for line in lineNames:                    # 对于每一段
    for name1 in line:                    
        for name2 in line:                # 每段中的任意两个关键词
            if name1 == name2:
                continue
            if relationships[name1].get(name2) is None:        # 若两关键词尚未同时出现则新建项
                relationships[name1][name2]= 1
            else:
                relationships[name1][name2] = relationships[name1][name2]+ 1        # 两个关键词共同出现次数加 1

In [8]:
with codecs.open("ehikoo_node.txt", "w", "gbk") as f:
    f.write("Id Label Weight\r\n")
    for name, times in names.items():
        f.write(name + " " + name + " " + str(times) + "\r\n")

with codecs.open("ehikoo_edge.txt", "w", "gbk") as f:
    f.write("Source Target Weight\r\n")
    for name, edges in relationships.items():
        for v, w in edges.items():
            if w > 3:
                f.write(name + " " + v + " " + str(w) + "\r\n")